# Polarization In Yeast
***
Reactions could not be completed from the slides (74 - 82)
***
## Setup the Environment
***

In [ ]:
import spatialpy

***
## Create the Polarization In Yeast Model
***

In [ ]:
def create_polarizationinyeast(parameter_values=None):
    model = spatialpy.Model(name='Polarization-In-Yeast')

    # Define Domain Type IDs as constants of the Model
    model.CYTOPLASM = 'Cytoplasm'
    model.MEMBRANE = 'Membrane'

    # Domain
    domain = spatialpy.Domain(
        0, (-0.9985541604477006, 0.9947062116569865), (-0.9923297008495786, 0.9902528598554616),
        (-0.9929082749587046, 1), rho0=1, c0=10, P0=100, gravity=None
    )

    import os
    home = os.path.expanduser('~')
    # Domain Lattices
    ipa_lattice1 = spatialpy.XMLMeshLattice(
        os.path.join(home, 'V2-Spatial-Examples/Yeast-Polarization.proj/Domain-Files/polarization.xml'),
        subdomain_file=os.path.join(home, 'V2-Spatial-Examples/Yeast-Polarization.proj/Domain-Files/polarization_subdomains.txt')
    )

    # Domain Actions
    domain.add_fill_action(
        lattice=ipa_lattice1, enable=True, apply_action=True,
        mass=1, vol=1, rho=1, nu=0, c=10, fixed=False
    )

    model.add_domain(domain, allow_all_types=True)

    model.staticDomain = True

    # Variables
    Cdc42c = spatialpy.Species(name='Cdc42c', diffusion_coefficient=10, restrict_to=[model.CYTOPLASM, model.MEMBRANE])
    Cdc42m = spatialpy.Species(name='Cdc42m', diffusion_coefficient=0.0053, restrict_to=[model.MEMBRANE])
    model.add_species([
        Cdc42c, Cdc42m
    ])

    # Initial Conditions
    Cdc42c_ic = spatialpy.ScatterInitialCondition(species='Cdc42c', count=300, types=[model.CYTOPLASM])
    Cdc42m_ic = spatialpy.ScatterInitialCondition(species='Cdc42m', count=100, types=[model.MEMBRANE])
    model.add_initial_condition([
        Cdc42c_ic, Cdc42m_ic
    ])

    # Parameters
    kon = spatialpy.Parameter(name='kon', expression='1.6666666667e-05')
    koff = spatialpy.Parameter(name='koff', expression='0.15')
    kfb = spatialpy.Parameter(name='kfb', expression='0.166666666666667')
    model.add_parameter([
        kon, koff, kfb
    ])

    # Reactions
    r1 = spatialpy.Reaction(
        name='r1', rate='kon', restrict_to=[model.CYTOPLASM, model.MEMBRANE],
        reactants={'Cdc42c': 1}, products={'Cdc42m': 1}
    )
    r2 = spatialpy.Reaction(
        name='r2', rate='koff', restrict_to=[model.CYTOPLASM, model.MEMBRANE],
        reactants={'Cdc42m': 1}, products={'Cdc42c': 1}
    )
    r3 = spatialpy.Reaction(
        name='r3', rate='kfb', restrict_to=[model.MEMBRANE],
        reactants={'Cdc42c': 1, 'Cdc42m': 1}, products={'Cdc42m': 2}
    )
    model.add_reaction([
        r1, r2, r3
    ])

    # Timespan
    tspan = spatialpy.TimeSpan.arange(10, t=5000, timestep_size=1)
    model.timespan(tspan)
    return model

### Instantiate the model

In [ ]:
model = create_polarizationinyeast()

In [ ]:
model.domain

***
## Simulation Parameters
***

In [ ]:
def configure_simulation():
    kwargs = {
        # 'number_of_trajectories': 1,
        # 'seed': None,
    }
    return kwargs

***
## Run the Simulation
***

In [ ]:
kwargs = configure_simulation()
results = model.run(**kwargs)

***
## Visualization
***

In [ ]:
results.plot_species('Cdc42c', animated=True, width='auto', height='auto')